In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('../../../datasets/level2/DS_Sample_1.csv')

In [17]:
card1 = df.fillna({'CREDIT_LIMIT': df['CREDIT_LIMIT'].mean(), 'MINIMUM_PAYMENTS': df['MINIMUM_PAYMENTS'].mean()})
card1.isna().sum()

CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

In [29]:
card1.groupby('TENURE')['BALANCE', 'CREDIT_LIMIT'].corr().round(2)

C:\Users\Lee\AppData\Local\Temp\ipykernel_41360\2734815348.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  card1.groupby('TENURE')['BALANCE', 'CREDIT_LIMIT'].corr().round(2)


BALANCE  CREDIT_LIMIT
TENURE                                    
6.0    BALANCE          1.00          0.87
       CREDIT_LIMIT     0.87          1.00
7.0    BALANCE          1.00          0.95
       CREDIT_LIMIT     0.95          1.00
8.0    BALANCE          1.00          0.82
       CREDIT_LIMIT     0.82          1.00
9.0    BALANCE          1.00          0.09
       CREDIT_LIMIT     0.09          1.00
10.0   BALANCE          1.00          0.29
       CREDIT_LIMIT     0.29          1.00
11.0   BALANCE          1.00          0.38
       CREDIT_LIMIT     0.38          1.00
12.0   BALANCE          1.00          0.46
       CREDIT_LIMIT     0.46          1.00

In [76]:
card1.iloc[:, 1:].head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,cluster
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0,0
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0,0


In [36]:
card1.iloc[:, 1:].std()

BALANCE                             2795.390979
BALANCE_FREQUENCY                      0.210538
PURCHASES                           3524.515043
ONEOFF_PURCHASES                    2819.795174
INSTALLMENTS_PURCHASES              1272.592078
CASH_ADVANCE                        2730.477215
PURCHASES_FREQUENCY                    0.406850
ONEOFF_PURCHASES_FREQUENCY             0.335454
PURCHASES_INSTALLMENTS_FREQUENCY       0.407137
CASH_ADVANCE_FREQUENCY                 0.208752
CASH_ADVANCE_TRX                       7.647122
PURCHASES_TRX                         31.294669
CREDIT_LIMIT                        4576.527947
PAYMENTS                            4251.971043
MINIMUM_PAYMENTS                    2610.145241
PRC_FULL_PAYMENT                       0.278915
TENURE                                 1.016693
dtype: float64

In [43]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [45]:
scaler = StandardScaler()
scaler.fit(card1.iloc[:, 1:])
card1_std = pd.DataFrame(scaler.transform(card1.iloc[:, 1:]), columns=card1.iloc[:, 1:].columns)
card1_std.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


In [66]:
def kmeans(k):
    model = KMeans(n_clusters=k, random_state=1234).fit(X=card1_std, y=card1_std['ONEOFF_PURCHASES'])
    pred = model.predict(card1_std)
    card1_kmeans = card1
    card1_kmeans['cluster'] = pred
    mean = card1_kmeans.groupby('cluster')['ONEOFF_PURCHASES'].mean().round(2)
    score = silhouette_score(card1_kmeans, labels=card1_kmeans['cluster'], random_state=1234)
    print(k, score, mean)

In [67]:
for i in range(2, 6):
    kmeans(i)

2 0.37470505948267385 cluster
0     340.23
1    3946.19
Name: ONEOFF_PURCHASES, dtype: float64
3 0.04690160753299618 cluster
0     255.39
1    8104.71
2    1103.22
Name: ONEOFF_PURCHASES, dtype: float64
4 0.06989359018950245 cluster
0     266.43
1    8894.33
2     523.65
3    1144.02
Name: ONEOFF_PURCHASES, dtype: float64
5 0.04636315119410011 cluster
0      272.86
1     4358.69
2      653.77
3      560.75
4    25404.44
Name: ONEOFF_PURCHASES, dtype: float64


In [80]:
card1_train = card1.loc[card1['CUST_ID'] % 4 != 0, :]
card1_test = card1.loc[card1['CUST_ID'] % 4 == 0, :]

print(card1_train.size, card1_test.size)

13536 4464


In [81]:
card1.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001.0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002.0,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [71]:
from sklearn.tree import DecisionTreeRegressor

In [114]:
X_cols = [i for i in range(1, 18)]
X_cols.remove(4)

model = DecisionTreeRegressor(max_depth=5, random_state=1234).fit(X=card1_train.iloc[:, X_cols], y=card1_train['ONEOFF_PURCHASES'])
pred = model.predict(card1_test.iloc[:, X_cols])

result = pd.DataFrame({'true': card1_test['ONEOFF_PURCHASES'], 'pred': pred})
result.head()

,true,pred
3,1499.00,1426.623483
7,0.00,0.000000
11,1492.18,1426.623483
15,0.00,497.815946
19,0.00,0.000000


In [142]:
sum = 0

for i in result.values:
    sum += (i[0] - i[1]) ** 2

B = round((sum / result.shape[0]) ** (1 / 2))
B

2316